Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'

# If you're working locally:
else:
    DATA_PATH = '../data/'

# Module Project: Regression II

In this project, you'll continue working with the New York City rent dataset you used in the last module project.

## Directions

The tasks for this project are as follows:

- **Task 1:** Import `csv` file using `wrangle` function.
- **Task 2:** Conduct exploratory data analysis (EDA), and modify `wrangle` function to engineer two new features.
- **Task 3:** Split data into feature matrix `X` and target vector `y`.
- **Task 4:** Split feature matrix `X` and target vector `y` into training and test sets.
- **Task 5:** Establish the baseline mean absolute error for your dataset.
- **Task 6:** Build and train a `Linearregression` model.
- **Task 7:** Calculate the training and test mean absolute error for your model.
- **Task 8:** Calculate the training and test $R^2$ score for your model.
- **Stretch Goal:** Determine the three most important features for your linear regression model.

**Note**

You should limit yourself to the following libraries for this project:

- `matplotlib`
- `numpy`
- `pandas`
- `sklearn`

# I. Wrangle Data

In [2]:
# Step 1 is always to import your libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def wrangle(filepath):
    df = pd.read_csv(filepath,
                     parse_dates=['created'],
                     index_col=['created'])
    
    # Remove the most extreme 1% prices,
    # the most extreme .1% latitudes, &
    # the most extreme .1% longitudes
    df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
            (df['price'] <= np.percentile(df['price'], 99.5)) & 
            (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
            (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
            (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
            (df['longitude'] <= np.percentile(df['longitude'], 99.95))]
    
    # Drop rows with NaN values
    df.dropna(axis=0, inplace=True)

    # New Features for the DataSet
    # TOTAL ROOMS
    df['total_rooms'] =  df['bathrooms'] + df['bedrooms']

    # ALL PETS ALLOWED
    df.loc[(df['dogs_allowed'] == 1) | (df['cats_allowed'] == 1), 'pets_allowed'] = 1
    df.loc[(df['dogs_allowed'] != 1) & (df['cats_allowed'] != 1), 'pets_allowed'] = 0

    # LUXURIES
    df.loc[(df['hardwood_floors'] == 1) & (df['dishwasher'] == 1) & (df['laundry_in_unit'] == 1), 'much_luxury'] = 1
    df.loc[(df['hardwood_floors'] != 1) | (df['dishwasher'] != 1) | (df['laundry_in_unit'] != 1), 'much_luxury'] = 0

    return df

filepath = DATA_PATH + 'apartments/renthop-nyc.csv'

**Task 1:** Add the following functionality to the above `wrangle` function.

- The `'created'` column will parsed as a `DateTime` object and set as the `index` of the DataFrame. 
- Rows with `NaN` values will be dropped.

Then use your modified function to import the `renthop-nyc.csv` file into a DataFrame named `df`.

In [4]:
df = wrangle(filepath)

df.head()

,bathrooms,bedrooms,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,total_rooms,pets_allowed,much_luxury
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0.0,0.0
2016-06-12 12:19:27,1.0,2,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,1.0,0.0
2016-04-17 03:26:41,1.0,1,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0
2016-04-18 02:22:02,1.0,1,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0
2016-04-28 01:32:41,1.0,4,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,0.0,0.0


In [5]:
# Check to make sure there are no NaN values in the DataFrame
df.isna().sum().sum()

0

**Task 2:** Using your `pandas` and dataviz skills decide on two features that you want to engineer for your dataset. Next, modify your `wrangle` function to add those features. 

**Note:** You can learn more about feature engineering [here](https://en.wikipedia.org/wiki/Feature_engineering). Here are some ideas for new features:

- Does the apartment have a description?
- Length of description.
- Total number of perks that apartment has.
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths).

In [6]:
# Conduct your exploratory data analysis here, 
# and then modify the function above.
# print(df.nunique())

# Explore here to learn how these values are distributed
print(df['bathrooms'].value_counts())
print(df['bedrooms'].value_counts())

# Find your second && condition for Pets Allowed
print(df['dogs_allowed'].value_counts())


1.0     37816
2.0      7421
3.0       663
1.5       641
0.0       304
2.5       256
4.0        91
3.5        55
4.5         8
5.0         4
10.0        1
Name: bathrooms, dtype: int64
1    14987
2    14181
0     9003
3     7041
4     1793
5      209
6       43
8        2
7        1
Name: bedrooms, dtype: int64
0    26306
1    20954
Name: dogs_allowed, dtype: int64


# II. Split Data

**Task 3:** Split your DataFrame `df` into a feature matrix `X` and the target vector `y`. You want to predict `'price'`.

**Note:** In contrast to the last module project, this time you should include _all_ the numerical features in your dataset.

In [7]:
# Columns to not include
# Remove Redundant Variables that are included in your features
cols = ['description', 'display_address', 'street_address', 'interest_level', 'price', 'bedrooms', 'bathrooms', 'dogs_allowed', 'hardwood_floors', 'dishwasher', 'laundry_in_unit']

X = df.drop(columns = cols)
y = df['price']

# Check the shape of both X and y 
print(X.shape)
print(y.shape)

(47260, 25)
(47260,)


**Task 4:** Split `X` and `y` into a training set (`X_train`, `y_train`) and a test set (`X_test`, `y_test`).

- Your training set should include data from April and May 2016. 
- Your test set should include data from June 2016.

In [8]:
# Create an 80% cutoff value here using April and May cutoff months
cutoff = (X.index.month == 4) | (X.index.month == 5)

X_train, y_train = X.loc[cutoff], y.loc[cutoff]
X_test, y_test = X.loc[~cutoff], y.loc[~cutoff]

# Print X train and X test to make sure your code is correct
print(X_train)
print(X_test)

                     latitude  longitude  ...  pets_allowed  much_luxury
created                                   ...                           
2016-04-17 03:26:41   40.7388   -74.0018  ...           0.0          0.0
2016-04-18 02:22:02   40.7539   -73.9677  ...           0.0          0.0
2016-04-28 01:32:41   40.8241   -73.9493  ...           0.0          0.0
2016-04-19 04:24:47   40.7429   -74.0028  ...           0.0          0.0
2016-04-27 03:19:56   40.8012   -73.9660  ...           1.0          0.0
...                       ...        ...  ...           ...          ...
2016-04-22 15:44:11   40.7296   -73.9869  ...           1.0          0.0
2016-04-04 18:22:34   40.7102   -74.0163  ...           1.0          0.0
2016-04-16 02:13:40   40.7601   -73.9900  ...           1.0          0.0
2016-04-08 02:13:33   40.7066   -74.0101  ...           1.0          0.0
2016-04-12 02:48:07   40.8699   -73.9172  ...           0.0          0.0

[30806 rows x 25 columns]
                     lat

# III. Establish Baseline

**Task 5:** Since this is a **regression** problem, you need to calculate the baseline mean absolute error for your model. First, calculate the mean of `y_train`. Next, create a list `y_pred` that has the same length as `y_train` and where every item in the list is the mean. Finally, use `mean_absolute_error` to calculate your baseline.

In [9]:
y_mean = y_train.mean()

# Import sklearn 
from sklearn.metrics import mean_absolute_error

y_pred = [y_mean] * len(y_train)

baseline_mae = mean_absolute_error(y_train, y_pred)
print('Baseline MAE:', baseline_mae)

Baseline MAE: 1202.398300781848


# IV. Build Model

**Task 6:** Build and train a `LinearRegression` model named `model` using your feature matrix `X_train` and your target vector `y_train`.

In [10]:
# Step 1: Import predictor class
from sklearn.linear_model import LinearRegression

# Step 2: Instantiate predictor
model = LinearRegression()

# Step 3: Fit predictor on the (training) data

model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# V. Check Metrics

**Task 7:** Calculate the training and test mean absolute error for your model.

In [11]:
# Use your train and test models to find predictions for y
y1_train_pred = model.predict(X_train)
y1_test_pred = model.predict(X_test)

# Now use these values to calculate the train and test Mean Absolute Error
training_mae = mean_absolute_error(y_train, y1_train_pred)
test_mae = mean_absolute_error(y_test, y1_test_pred)

print('Training MAE:', training_mae)
print('Test MAE:', test_mae)

Training MAE: 747.6761656314386
Test MAE: 759.2499182895607


**Task 8:** Calculate the training and test $R^2$ score for your model.

In [12]:
# Import r2
from sklearn.metrics import r2_score

training_r2 = r2_score(y_train, y1_train_pred)
test_r2 = r2_score(y_test, y1_test_pred)

print('Training MAE:', training_r2)
print('Test MAE:', test_r2)

Training MAE: 0.5611133939523953
Test MAE: 0.5654439752973328


# VI. Communicate Results

**Stretch Goal:** What are the three most influential coefficients in your linear model? You should consider the _absolute value_ of each coefficient, so that it doesn't matter if it's positive or negative.

In [13]:
# Step 1 is to list all of the coefficients
coef = model.coef_
print(coef)

# Step 2 is to list all column headers
col_names = X.columns
print(col_names)

# Step 3 is to create a new DataFrame to compare coefficients
df_stretch = pd.DataFrame(coef, index= col_names)

# Print the DataFrame Header to check
print(df_stretch.head())

[  1809.28167967 -13748.63461857    154.82197234    206.54477786
    633.95362426   -261.57407203   -287.95401078    198.45666836
    -50.34963432   -223.06658968   -127.78742267    267.80553396
   -428.42553857    -29.18463245    106.96723285    -78.85890897
    206.98002177    330.00812969    273.37968309    -55.7407297
    270.69152543   -112.33975853    801.5823805    -116.01718506
    504.05708399]
Index(['latitude', 'longitude', 'elevator', 'cats_allowed', 'doorman',
       'no_fee', 'laundry_in_building', 'fitness_center', 'pre-war',
       'roof_deck', 'outdoor_space', 'dining_room', 'high_speed_internet',
       'balcony', 'swimming_pool', 'new_construction', 'terrace', 'exclusive',
       'loft', 'garden_patio', 'wheelchair_access', 'common_outdoor_space',
       'total_rooms', 'pets_allowed', 'much_luxury'],
      dtype='object')
                         0
latitude       1809.281680
longitude    -13748.634619
elevator        154.821972
cats_allowed    206.544778
doorman     

In [14]:
# Now that we have our dataframe 
df_stretch.abs()

df_stretch[0].abs().sort_values(ascending= False)

longitude               13748.634619
latitude                 1809.281680
total_rooms               801.582380
doorman                   633.953624
much_luxury               504.057084
high_speed_internet       428.425539
exclusive                 330.008130
laundry_in_building       287.954011
loft                      273.379683
wheelchair_access         270.691525
dining_room               267.805534
no_fee                    261.574072
roof_deck                 223.066590
terrace                   206.980022
cats_allowed              206.544778
fitness_center            198.456668
elevator                  154.821972
outdoor_space             127.787423
pets_allowed              116.017185
common_outdoor_space      112.339759
swimming_pool             106.967233
new_construction           78.858909
garden_patio               55.740730
pre-war                    50.349634
balcony                    29.184632
Name: 0, dtype: float64

It seems that the largest coefficients for this dataset are Longitude, Latitude, and Total Rooms